In [1]:
import pandas as pd

In [4]:
PATH = "../data/ml-100k/"

In [11]:
m=pd.read_csv(
    PATH + 'u.data',
    sep='\t',
    encoding="latin1",
    names=['user_id', 'item_id', 'rating', 'timestamp']
    )[['user_id', 'item_id', 'rating']]
m.head(5)

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [13]:
m.dtypes

user_id    int64
item_id    int64
rating     int64
dtype: object